In [23]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from requests.exceptions import ConnectionError
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
import time
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [25]:
def check_within_day(dt):
    current_datetime = datetime.now()
    time_difference = current_datetime - dt
    if time_difference.total_seconds() < 86400:
        return True
    else:
        return False

In [126]:
def download(url, save_path):
    response = requests.get(url, stream=True)
    # Check if the request was successful
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)
    else:
        raise ConnectionError()


In [118]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.reddit.com/r/Myanmarcombatfootage/new")
# Get scroll height
day_loaded = False
num_results = 0
post_info = []
while not day_loaded:
    html = driver.page_source
    soup = bs(html, 'html.parser')
    results = soup.find_all('shreddit-post')
    for result in results[num_results:]:
        flair = result.find('span', class_='bg-tone-4')
        if flair:
            flair_text = flair.text.split('\n')[1][6:]
        else:
            flair_text = "no flair"

        shreddit_post = bs(str(result).split('\n')[0], 'html.parser').find()
        author = shreddit_post['author']
        title = shreddit_post['post-title']
        score = shreddit_post.get('score', 'hidden')
        link = shreddit_post['content-href']
        timestamp = shreddit_post['created-timestamp']
        post_type = shreddit_post['post-type']

        timestamp = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f%z")
        dt = timestamp.replace(tzinfo=None)
        if not check_within_day(dt):
            day_loaded = True
            break
        else:
            video = None
            audio = None

            post_info.append({
                "author": author,
                "title": title,
                "score": score,
                "content_link": link,
                "dt": dt,
                "flair": flair_text,
                "post_type": post_type,
            })
    
    # get previous result length to skip past checked results
    num_results = len(results)
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
driver.quit()
post_info


[{'author': 'irishtimezones',
  'title': 'Anti-Junta forces attacking the Launglon Myoma Police Station on September 21st, 6:23AM local time. It is reported two members of the security forces were eliminated. Also, quite an interesting video due to them using a guncam.',
  'score': '10',
  'content_link': 'https://v.redd.it/icqzf7lbsmpb1',
  'dt': datetime.datetime(2023, 9, 21, 16, 3, 25, 419000),
  'flair': 'PDF',
  'post_type': 'video',
  'video': 'https://v.redd.it/icqzf7lbsmpb1/DASH_640.mp4?source=fallback',
  'audio': 'https://v.redd.it/icqzf7lbsmpb1/DASH_AUDIO_128.mp4'},
 {'author': 'irishtimezones',
  'title': 'Anti-junta Forces attacking Myanmar army columns belonging to the 101st Light Infantry Division and several casualties, including atleast 1 fatality, were reported in Pakokku, Magwe, 21 September.',
  'score': '10',
  'content_link': 'https://v.redd.it/dbcipfiggmpb1',
  'dt': datetime.datetime(2023, 9, 21, 14, 52, 21, 899000),
  'flair': 'Drone ',
  'post_type': 'video',


In [129]:
for post in post_info:
    title = post['title']
    link = post['content_link']
    print(post['title'])

    if post['post_type'] == 'video':
        try:
            video_720 = f'{link}/DASH_720.mp4?source=fallback'
            download(video_720, f'downloads/{title}_video_720.mp4')
        except ConnectionError as e:
            print(e.args)
            video_360 = f'{link}/DASH_360.mp4?source=fallback'
            download(video_360, f'downloads/{title}_video_360.mp4')
        finally:
            print(video_720, video_360)

        audio_128 = f'{link}/DASH_AUDIO_128.mp4'
        download(audio_128, f'downloads/{title}_audio_128.mp4')
    
    else:
        print(post['content_link'])

    time.wait(1)

Anti-Junta forces attacking the Launglon Myoma Police Station on September 21st, 6:23AM local time. It is reported two members of the security forces were eliminated. Also, quite an interesting video due to them using a guncam.
()
https://v.redd.it/icqzf7lbsmpb1/DASH_720.mp4?source=fallback https://v.redd.it/icqzf7lbsmpb1/DASH_360.mp4?source=fallback
Anti-junta Forces attacking Myanmar army columns belonging to the 101st Light Infantry Division and several casualties, including atleast 1 fatality, were reported in Pakokku, Magwe, 21 September.
https://v.redd.it/dbcipfiggmpb1/DASH_720.mp4?source=fallback https://v.redd.it/icqzf7lbsmpb1/DASH_360.mp4?source=fallback
Greener GP and M1 Carbines captured from a Pyu Saw Htee outpost in Gangaw township, Magway Region.
https://www.reddit.com/gallery/16ogpug
Myanmar Army handing out captured Type 81s and old vintage rifles from storage to Pro-military militias in Bago Rigions.
https://www.reddit.com/gallery/16ogoog
Weapons and munitions capture 